In [3]:
import time
start_time = time.time()
import numpy as np
import pandas as pd
import nltk, re, pprint
from __future__ import division

In [4]:
# read the training data
df_train = pd.read_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/Data/train.csv", encoding="ISO-8859-1") #update here
df_test = pd.read_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/Data/test.csv", encoding="ISO-8859-1") #update here
df_att = pd.read_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/Data/attributes.csv", encoding="ISO-8859-1") #update here
df_pro = pd.read_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/Data/product_descriptions.csv", encoding="ISO-8859-1") #update here

In [5]:
df_train_df = pd.DataFrame(df_train)
df_train_pro = pd.DataFrame(df_pro)
df_test_df = pd.DataFrame(df_test)

In [6]:
result = pd.merge(df_train_df, df_train_pro, on='product_uid')
result_test = pd.merge(df_test_df, df_train_pro, on='product_uid')

In [8]:
result.head(1)

,id,product_uid,product_title,search_term,relevance,product_description
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3,"Not only do angles make joints stronger, they ..."


In [7]:
# define the function for splitting the words
def split_search(value) :
    res = [w.lower() for w in (str(value).split())]
    return res

In [8]:
# add a new column called search_split
ss = result["search_term"].apply(split_search) 
result['search_split'] = pd.Series(ss, index=result.index)
ss = result_test["search_term"].apply(split_search) 
result_test['search_split'] = pd.Series(ss, index=result_test.index)

In [9]:
# help to split the string
def split_search(value) :
    res = [w.lower() for w in (str(value).split())]
    return res

# calculate the word length for each of the searching words
# remove the word that has length 1
nrow = result.shape[0]
new_search_split = [0]*nrow
for index, row in result.iterrows():
    each_word_len = [len(w) for w in row["search_split"]]
    row["search_split"] = [word for word in row["search_split"] if len(word) > 1]
    new_search_split[index] = row["search_split"]
    each_word_len = [mm for mm in each_word_len if mm > 1]

# add a new column called search_split
result['new_search_split'] = pd.Series(new_search_split, index=result.index)

# help to split the string for test data
def split_search(value) :
    res = [w.lower() for w in (str(value).split())]
    return res

# calculate the word length for each of the searching words
# remove the word that has length 1

nrow = result_test.shape[0]
new_search_split = [0]*nrow
for index, row in result_test.iterrows():
    each_word_len = [len(str(w)) for w in row["search_split"]]
    row["search_split"] = [word for word in row["search_split"] if len(word) > 1]
    new_search_split[index] = row["search_split"]
    each_word_len = [mm for mm in each_word_len if mm > 1]

# add a new column called search_split
result_test['new_search_split'] = pd.Series(new_search_split, index=result_test.index)

In [10]:
# find how many words in the product description match the searching query 
nrow = result.shape[0]
desc_match_word = [0]*nrow
for index, row in result.iterrows():
    ll = [len(re.findall(w, row["product_description"])) for w in row["new_search_split"]]
    desc_match_word[index] = sum(ll)

# add a new column called description match words
result['desc_match_word'] = pd.Series(desc_match_word, index=result.index)


In [11]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [12]:
#result_test["product_description"][73024] = result_test["product_description"][1]
#result_test["new_search_split"][73024] = result_test["new_search_split"][1]
#118900
# 118901
# find how many words in the product description match the searching query 
nrow = result_test.shape[0]
desc_match_word = [0]*nrow
for index, row in result_test.iterrows():    
    ll = [len(re.findall(w, row["product_description"])) for w in row["new_search_split"]]
    print (ll)
    desc_match_word[index] = sum(ll)
result_test['desc_match_word'] = pd.Series(desc_match_word, index=result_test.index)


In [13]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [14]:
# find how many words match in the product title
nrow = result.shape[0]
title_match_word = [0]*nrow
for index, row in result.iterrows():
    ll = [len(re.findall(w, str(row["product_title"]).lower())) for w in row["new_search_split"]]
    print (ll)
    print(index)
    title_match_word[index] = sum(ll)

# add a new column called description match words
result['title_match_word'] = pd.Series(title_match_word, index=result.index)

# test data
# find how many words match in the product title
nrow = result_test.shape[0]
title_match_word = [0]*nrow
for index, row in result_test.iterrows():
    ll = [len(re.findall(w, str(row["product_title"]).lower())) for w in row["new_search_split"]]
    print (ll)
    print(index)
    title_match_word[index] = sum(ll)

# add a new column called description match words
result_test['title_match_word'] = pd.Series(title_match_word, index=result_test.index)


In [15]:
# build random forest model

from sklearn.ensemble import RandomForestClassifier
cols = ['desc_match_word', 'title_match_word'] 
colsRes = ['relevance']
trainArr = result.as_matrix(cols) #training array
trainRes = result.as_matrix(colsRes) # training results
rf = RandomForestClassifier(n_estimators=100) # initialize
rf.fit(trainArr, trainRes) # fit the data to the algorithm


RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0)

In [18]:
testArr = result_test.as_matrix(cols)
results = rf.predict(testArr)
relevance = results
relevance.shape

(166693,)

In [19]:
index = list(range(166694))[1:]

In [20]:
df = pd.DataFrame(index = index, data=relevance)
df

,0
1,2.33
2,2.33
3,3.00
4,3.00
5,3.00
6,2.33
7,2.67
8,3.00
9,3.00
10,3.00


In [772]:
df.to_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/submission/sub_4_20.csv")

## April 23 Analysis

Get rid of the stop words from the product title and the description:

In [16]:
# add the two new columns: product title and the product description, no stop words involved
from nltk.corpus import stopwords
# ...
filtered_product_title = [word for word in result["product_title"] if word not in stopwords.words('english')]
result['filtered_product_title'] = pd.Series(filtered_product_title, index=result.index)

filtered_product_description = [word for word in result["product_description"] if word not in stopwords.words('english')]
result['filtered_product_description'] = pd.Series(filtered_product_description, index=result.index)

#product_description 

In [17]:
# add the two new columns: product title and the product description, no stop words involved
from nltk.corpus import stopwords
# ...
filtered_product_title = [word for word in result_test["product_title"] if word not in stopwords.words('english')]
result_test['filtered_product_title'] = pd.Series(filtered_product_title, index=result_test.index)

filtered_product_description = [word for word in result_test["product_description"] if word not in stopwords.words('english')]
result_test['filtered_product_description'] = pd.Series(filtered_product_description, index=result_test.index)

#product_description 

Get the word length of the product description and the product title

In [18]:
product_descr_len = result["filtered_product_description"].apply(split_search).apply(len)
#result["product_description"][1]
result['product_descr_len'] = pd.Series(product_descr_len, index=result.index)

product_title_len = result["filtered_product_title"].apply(split_search).apply(len)
#result["product_description"][1]
result['product_title_len'] = pd.Series(product_title_len, index=result.index)


In [19]:
#### for the test data
product_descr_len = result_test["filtered_product_description"].apply(split_search).apply(len)
#result["product_description"][1]
result_test['product_descr_len'] = pd.Series(product_descr_len, index=result_test.index)

product_title_len = result_test["filtered_product_title"].apply(split_search).apply(len)
#result["product_description"][1]
result_test['product_title_len'] = pd.Series(product_title_len, index=result_test.index)

The percentage of match for production description and product title

In [20]:
desc_match_perc = result["desc_match_word"]/result["product_descr_len"]
title_match_perc = result["title_match_word"]/result["product_title_len"]
result['desc_match_perc'] = pd.Series(desc_match_perc, index=result.index)
result['title_match_perc'] = pd.Series(title_match_perc, index=result.index)

In [21]:
# test data 
desc_match_perc = result_test["desc_match_word"]/result_test["product_descr_len"]
title_match_perc = result_test["title_match_word"]/result_test["product_title_len"]
result_test['desc_match_perc'] = pd.Series(desc_match_perc, index=result_test.index)
result_test['title_match_perc'] = pd.Series(title_match_perc, index=result_test.index)

In [21]:
from sklearn.ensemble import RandomForestClassifier
cols = ['desc_match_word', 'title_match_word', 'desc_match_perc', 'title_match_perc'] 
colsRes = ['relevance']
trainArr = result.as_matrix(cols) #training array
trainRes = result.as_matrix(colsRes) # training results
rf = RandomForestClassifier(n_estimators=100) # initialize
rf.fit(trainArr, trainRes) # fit the data to the algorithm


RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0)

In [34]:
testArr = result_test.as_matrix(cols)
results = rf.predict(testArr)
relevance = results
relevance.shape

df = pd.DataFrame(index = result_test['id'], data=relevance)
df


,0
id,
1,2.33
4,2.33
5,3.00
6,2.00
7,2.67
8,2.33
10,2.67
11,3.00
12,3.00


In [35]:
df.to_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/submission/sub_4_20.csv")

### April 28, add the xgboost

https://www.kaggle.com/cbrogan/titanic/xgboost-example-python/code


In [27]:
result.head(2)

,id,product_uid,product_title,search_term,relevance,product_description,search_split,new_search_split,desc_match_word,title_match_word,filtered_product_title,filtered_product_description,product_descr_len,product_title_len,desc_match_perc,title_match_perc
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.0,"Not only do angles make joints stronger, they ...","[angle, bracket]","[angle, bracket]",3,1,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they ...",129,4,0.023256,0.25
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.5,"Not only do angles make joints stronger, they ...","[l, bracket]",[bracket],0,0,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they ...",129,4,0.000000,0.00


In [28]:
result_test.head(2)

,id,product_uid,product_title,search_term,product_description,search_split,new_search_split,desc_match_word,title_match_word,filtered_product_title,filtered_product_description,product_descr_len,product_title_len,desc_match_perc,title_match_perc
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket,"Not only do angles make joints stronger, they ...","[90, degree, bracket]","[90, degree, bracket]",1,0,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they ...",129,4,0.007752,0
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets,"Not only do angles make joints stronger, they ...","[metal, l, brackets]","[metal, brackets]",0,0,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they ...",129,4,0.000000,0


In [22]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [23]:
# append the features from train and test together
features = ['desc_match_word', 'title_match_word', 'desc_match_perc', 'title_match_perc']
test_add_train = result[features].append(result_test[features])

In [24]:
# Prepare the inputs for the model
train_part = test_add_train[0:result.shape[0]].as_matrix()
test_part = test_add_train[result.shape[0]::].as_matrix()
relevance_score = result['relevance']

In [33]:
gbm = xgb.XGBClassifier(max_depth=5, n_estimators=300, learning_rate=0.05).fit(train_part, relevance_score)
predictions = gbm.predict(test_part)

In [34]:
# Kaggle needs the submission to have a certain format;
# see https://www.kaggle.com/c/titanic-gettingStarted/download/gendermodel.csv
# for an example of what it's supposed to look like.
submission = pd.DataFrame({ 'id': result_test['id'],
                            'relevance': predictions })
submission.to_csv("4_28_sub.csv", index=False)


In [35]:
result.head(1)

,id,product_uid,product_title,search_term,relevance,product_description,search_split,new_search_split,desc_match_word,title_match_word,filtered_product_title,filtered_product_description,product_descr_len,product_title_len,desc_match_perc,title_match_perc
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3,"Not only do angles make joints stronger, they ...","[angle, bracket]","[angle, bracket]",3,1,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they ...",129,4,0.023256,0.25


### Add the number matching feature

In [25]:
# drop the relevance score
result_wt_score = result.drop('relevance', 1)

In [26]:
whole = result_wt_score.append(result_test)

In [34]:
pd.merge(df_train_df, df_train_pro, on='product_uid')

,id,product_uid,product_title,search_term,relevance,product_description
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ..."
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ..."
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Update your bathroom with the Delta Vero Singl...
5,18,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,convection otr,3.00,Achieving delicious results is almost effortle...
6,20,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,microwave over stove,2.67,Achieving delicious results is almost effortle...
7,21,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,microwaves,3.00,Achieving delicious results is almost effortle...
8,23,100007,Lithonia Lighting Quantum 2-Light Black LED Em...,emergency light,2.67,The Quantum Adjustable 2-Light LED Black Emerg...
9,27,100009,House of Fara 3/4 in. x 3 in. x 8 ft. MDF Flut...,mdf 3/4,3.00,Get the House of Fara 3/4 in. x 3 in. x 8 ft. ...


In [27]:
# merge the two column in attrivute together
# attr: combine t
df_att["name_and_value"] = df_att["name"].map(str) + ' ' + df_att["value"]

In [28]:
# further handle the attribute data
def f(x):
    return x['name_and_value'].values

df_att_2 = df_att.groupby('product_uid').apply(f)

#series.name = 'metric'
df_att_2 = df_att_2.reset_index()
df_att_2.columns = ['product_uid', 'name_and_value']

# define the function for extracting the number from the value_and_name
def array_combine(ind) :
    val = ''.join(str(df_att_2['name_and_value'][ind]))
    return val

l=[]
for i in range(df_att_2.shape[0]):
    ind = array_combine(i)
    l.append(ind)
df_att_2['name_and_value_bind'] = pd.Series(l, index=df_att_2.index)


In [29]:
stor=[]
for x in range(0, df_att_2.shape[0]):
    ind = ",".join(str(i) for i in df_att_2['name_and_value'][x])
    indStr = ind
    stor.append(indStr)

In [30]:
df_att_2['att_final'] = pd.Series(stor, index=df_att_2.index)
# 43398, 4

In [31]:
whole_with_att = pd.merge(whole, df_att_2, how='left', on='product_uid')

In [32]:
def isNaN(num):
    return num != num

storNum=[]
for x in range(0, whole_with_att.shape[0]):
    if isNaN(whole_with_att['att_final'][x]) != True:
        ind = re.findall(r'[-+]?\d*\.\d+|\d+', whole_with_att['att_final'][x])
    else:
        ind = ['']
    storNum.append(ind)

In [33]:
whole_with_att['att_num'] = pd.Series(storNum, index=whole_with_att.index)

In [34]:
# add the new colum: length of number
length_num = whole_with_att['att_num'].apply(len)
whole_with_att['att_num_len'] = pd.Series(length_num, index=whole_with_att.index)

In [35]:
nrow = whole_with_att.shape[0]
title_match_word = [0]*nrow

for index, row in whole_with_att.iterrows():
    ll = [len(re.findall(w, str(row["att_num"]).lower())) for w in row["new_search_split"]]
    title_match_word[index] = sum(ll)
whole_with_att['att_num_match_len'] = pd.Series(title_match_word, index=whole_with_att.index)    

## xgboost

In [149]:
# append the features from train and test together
features = ['desc_match_word', 'title_match_word', 'desc_match_perc', 'title_match_perc', 'att_num_match_len']
# Prepare the inputs for the model
whole_with_feature = whole_with_att[features]
train_part = whole_with_feature[0:result.shape[0]].as_matrix()
test_part = whole_with_feature[result.shape[0]::].as_matrix()
relevance_score = result['relevance']

In [36]:
# you can tun the parameter here
# multi:softprob 
# reg:linear
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=5, learning_rate=0.01, objective="multi:softprob").fit(train_part, relevance_score)
predictions = gbm.predict(test_part)

In [162]:
predictions = gbm.predict(train_part)
sum((relevance_score-predictions)**2)/len(relevance_score)
# max_depth=5, n_estimators=300, learning_rate=0.25 0.43932219071903972


0.25350535461128881

In [163]:
# Kaggle needs the submission to have a certain format;
# see https://www.kaggle.com/c/titanic-gettingStarted/download/gendermodel.csv
# for an example of what it's supposed to look like.
predictions = gbm.predict(test_part)
submission = pd.DataFrame({ 'id': result_test['id'],
                            'relevance': predictions })
submission.to_csv("4_28_add_feature.csv", index=False)


In [164]:
whole_with_feature[result.shape[0]::].shape

(166693, 5)

## Generate more features

1. product_descr_len
2. product_title_len
3. desc_match_perc
4. title_match_perc
5. att_num_len
6. att_num_match_len
7. num_match_percent
8. new_search_len

In [37]:
nrow = whole_with_att.shape[0]
title_match_word = [0]*nrow
num_match_percent = whole_with_att['att_num_match_len']/whole_with_att['att_num_len']
whole_with_att['num_match_percent'] = pd.Series(num_match_percent, index=whole_with_att.index)    

In [38]:
new_search_len = whole_with_att['new_search_split'].apply(len)
whole_with_att['new_search_len'] = pd.Series(new_search_len, index=whole_with_att.index)    

In [39]:
# append the features from train and test together
features = ['product_descr_len', 'product_title_len', 'desc_match_perc', 'title_match_perc', 'att_num_len', 'att_num_match_len',
           'num_match_percent', 'new_search_len']
# Prepare the inputs for the model
whole_with_feature = whole_with_att[features]
train_part = whole_with_feature[0:result.shape[0]].as_matrix()
test_part = whole_with_feature[result.shape[0]::].as_matrix()
relevance_score = result['relevance']

In [40]:
# max_depth = 3, 5, 7,        9,  11,  13,  15
# n_estimators = 5, 10, 50, 100, 150, 200, 250, 300, 500
# learning_rate = 0.01, 0.05, 0.1, 0.15, 0.2
# objective: multi:softprob
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=5, learning_rate=0.01, objective="multi:softprob").fit(train_part, relevance_score)
predictions = gbm.predict(test_part)

In [41]:
from sklearn.metrics import mean_squared_error
results_train = gbm.predict(train_part)
RMSE = mean_squared_error(relevance_score, results_train)**0.5
RMSE


0.60163608668460578

In [42]:
predictions = gbm.predict(test_part)

In [43]:
# Kaggle needs the submission to have a certain format;
# see https://www.kaggle.com/c/titanic-gettingStarted/download/gendermodel.csv
# for an example of what it's supposed to look like.
predictions = gbm.predict(test_part)
submission = pd.DataFrame({ 'id': result_test['id'],
                            'relevance': predictions })
submission.to_csv("4_29.csv", index=False)


## Try the random forest

In [2]:
from sklearn.ensemble import RandomForestClassifier

#cols = ['desc_match_word', 'title_match_word', 'desc_match_perc', 'title_match_perc'] 

cols = ['product_descr_len', 'product_title_len', 'desc_match_perc', 'title_match_perc', 'att_num_len', 'att_num_match_len'
           'num_match_percent', 'new_search_len']
colsRes = ['relevance']

trainArr = whole_with_feature[0:result.shape[0]].as_matrix(cols) #training array
trainRes = result.as_matrix(colsRes) # training results


NameError: name 'whole_with_feature' is not defined

### NaN imputation

In [50]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
new_trainArr = imp.fit_transform(trainArr)

In [315]:
# n_estimators = 5, 10, 50, 100, 150
# min_samples_leaf = 5,10,50,100,150,500
rf = RandomForestClassifier(n_estimators= 300,  oob_score = True,random_state = 50, 
                            max_features = "auto", min_samples_leaf = 5) # initialize
rf.fit(new_trainArr, trainRes)

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=5,
            min_samples_split=2, n_estimators=300, n_jobs=1,
            oob_score=True, random_state=50, verbose=0)

In [316]:
testArr = whole_with_feature[result.shape[0]::].as_matrix(cols)
new_testArr = imp.fit_transform(testArr)

The fitted response variable for the training data

In [317]:
from sklearn.metrics import mean_squared_error
results_train = rf.predict(new_trainArr)
RMSE = mean_squared_error(trainRes, results_train)**0.5
RMSE

0.50980842307870622

In [318]:
results = rf.predict(new_testArr)
relevance = results
submission = pd.DataFrame({ 'id': result_test['id'],
                            'relevance': relevance})
submission.to_csv("4_28_50n_50min.csv", index=False)